# PUB Data Metric Functions (DMFs)
One method of monitoring data quality is through the uses data metric functions (DMFs), which include Snowflake-provided system DMFs and user-defined DMFs, to monitor the state and integrity of your data. 

You can use DMFs to measure key metrics, such as, but not limited to, freshness and counts that measure duplicates, NULLs, rows, and unique values.

In [ ]:
USE DATABASE PUB_DB;
CREATE SCHEMA GOVERNANCE;

## STEP 1: CREATING CUSTOM DMFs

### DQ 1. Must be a unique numeric field with 6 digits. No duplicates allowed.

This DMF returns the number of erroneous records that are duplicated or do not confirm to 6 digits.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_6DIG_DUPE_CHECK(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
    SELECT
      (SELECT COUNT(*) 
       FROM arg_t 
       WHERE LENGTH(arg_c1) != 6 
       OR arg_c1 IS NULL)
      +(SELECT COUNT(*) - COUNT(DISTINCT arg_c1) 
       FROM arg_t
       WHERE LENGTH(arg_c1) = 6 
       AND arg_c1 IS NOT NULL)
$$;

### DQ 2. Must contain only alphabetic characters and spaces. Cannot be empty. Maximum 250 characters.


This DMF returns the number of erroneous records that contain other non-alphabetic characters is empty or is more than 250 characters.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_ALPHABETIC_MAX250(
    arg_t TABLE(
        arg_c1 VARCHAR
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR TRIM(arg_c1) = ''
    OR LENGTH(arg_c1) > 250
    OR NOT REGEXP_LIKE(arg_c1, '^[A-Za-z ]+$')
$$

### DQ 3. Must follow Singapore ID format: 1 letter prefix (S/T), followed by 7 digits, ending with 1 letter suffix (e.g., S1234567D). Must be exactly 9 characters. Maximum 9 characters.




This DMF returns the number of erroneous records that do not follow Singapore ID format.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_SG_NRIC(
    arg_t TABLE(
        arg_c1 VARCHAR
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR NOT REGEXP_LIKE(arg_c1, '^[ST][0-9]{7}[A-Za-z]$')
    OR LENGTH(arg_c1) <> 9
$$;

### DQ 4. Must follow YYYY-MM format and represent a valid calendar month. Maximum 7 characters (e.g., 2025-01).

This DMF returns the number of erroneous records that do not follow the date format.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_YYYYMM_CHECK(
    arg_t TABLE(
        arg_c1 VARCHAR
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE LENGTH(arg_c1) <> 7
   OR NOT REGEXP_LIKE(arg_c1, '^\\d{4}-(0[1-9]|1[0-2])$')
$$;


### DQ 5. Must be a positive whole number representing liters. Cannot be negative.

This DMF returns the number of erroneous records that do not conform to the litres positive integer format

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_POSITIVE_WHOLE_NUMBER(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR arg_c1 < 0
    OR arg_c1 % 1 != 0
$$;


### DQ 6. Value between 0 and 100 inclusive

This DMF returns the number of erroneous records that do not conform to range of 0 to 100 inclusive.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_0_100_CHECK(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE
    arg_c1 IS NULL
    OR arg_c1 < 0
    OR arg_c1 > 100
$$;

### DQ 7. Must be a positive whole number. Expected range: 50 - 500.

This DMF returns the number of erroneous records that do not conform to whole numbers within the number range

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_50_TO_500(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR arg_c1 % 1 != 0
    OR arg_c1 < 50
    OR arg_c1 > 500
$$;

### DQ 8. Must be a numeric value with 2 decimal places. Normal range: 0.2 - 0.8 mg/L. Values > 1.0 should be flagged.

This DMF returns the number of erroneous records that do not conform to 2 decimal places and within the number range

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_LITRE_RANGE_FORMAT(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR ROUND(arg_c1, 2) != arg_c1
    OR arg_c1 > 1.00  
$$;

### DQ 9. Must be an integer between 1 (very dissatisfied) and 5 (very satisfied). Empty values are considered incomplete.

This DMF returns the number of erroneous records that do not conform to integer values between 1 and 5.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_SATISFACTION_SCORE(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR NOT IS_INTEGER(arg_c1) 
    OR arg_c1 <1
    OR arg_c1 >5
$$;

### DQ 10. Must be a positive monetary value. No more than 2 decimal places allowed in normal cases.

This DMF returns the number of erroneous records that do not conform to integer values between 1 and 5.

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_MAX2DECIMALS_POSITIVE(
    arg_t TABLE(
        arg_c1 NUMBER
    )
)
RETURNS NUMBER
AS
$$
SELECT COUNT(*)
FROM arg_t
WHERE 
    arg_c1 IS NULL
    OR arg_c1 <=0
    OR ROUND(arg_c1,2) != arg_c1
$$;

## STEP 2: Apply all the DMFs created above to the customer table:

In [ ]:
ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_0_100_CHECK
    ON (WATERQUALITYINDEX);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_50_TO_500
    ON (TDSPPM);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_6DIG_DUPE_CHECK
    ON (CUSTOMERID);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_ALPHABETIC_MAX250
    ON (CUSTOMERNAME);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_POSITIVE_WHOLE_NUMBER
    ON (WATERCONSUMPTIONLITERS);
    
ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_LITRE_RANGE_FORMAT
    ON (CHLORINELEVELMGL);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_MAX2DECIMALS_POSITIVE
    ON (PHLEVEL);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_SATISFACTION_SCORE
    ON (CUSTOMERFEEDBACK);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_SG_NRIC
    ON (NRIC);

ALTER TABLE PUB_DB.PUBLIC.CUSTOMER
  ADD DATA METRIC FUNCTION PUB_DB.GOVERNANCE.DQ_YYYYMM_CHECK
    ON (MONTH);

In [ ]:
ALTER TABLE PUB_DB.PUBLIC.CUSTOMER SET
  DATA_METRIC_SCHEDULE = '5 MINUTES';

In [ ]:
USE DATABASE PUB_DB;
USE SCHEMA PUBLIC;
SELECT *
  FROM TABLE(
    INFORMATION_SCHEMA.DATA_METRIC_FUNCTION_REFERENCES(
      REF_ENTITY_NAME => 'PUB_DB.PUBLIC.CUSTOMER',
      REF_ENTITY_DOMAIN => 'table'
    )
  );

## STEP 3: View DMF Results

In [ ]:
SELECT *
  FROM TABLE(SNOWFLAKE.LOCAL.DATA_QUALITY_MONITORING_RESULTS(
    REF_ENTITY_NAME => 'PUB_DB.PUBLIC.CUSTOMER',
    REF_ENTITY_DOMAIN => 'TABLE'));